## Setup

In [1]:
from pathlib import Path
from nanover.recording.reading import message_events_from_recording
from nanover.recording.writing import record_messages

In [2]:
def remove_keys_from_frame(frame, keys):
    for key in keys:
        frame.discard(key)
    return frame

## Config

In [4]:
from nanover.trajectory.frame_data import PARTICLE_FORCES, PARTICLE_VELOCITIES
keys_to_strip = { PARTICLE_FORCES, PARTICLE_VELOCITIES }

INPUT_PATH = Path("nanotube-example-recording.nanover.zip")
OUTPUT_PATH = Path("nanotube-example-recording--stripped.nanover.zip")

print("STRIP:", ", ".join(keys_to_strip), "\nINPUT:", INPUT_PATH, "\nOUTPUT:", OUTPUT_PATH)

STRIP: particle.forces, particle.velocities 
INPUT: nanotube-example-recording.traj 
OUTPUT: nanotube-example-recording-filtered.traj


## Run

In [5]:
# NBVAL_SKIP
frames = message_events_from_recording(INPUT_PATH)
filtered = ((timestamp, remove_keys_from_frame(frame, keys_to_strip)) for (timestamp, frame) in frames)
record_messages(OUTPUT_PATH, filtered)